In [6]:
!pip install -q grobid-client langchain openai faiss-cpu PyPDF2
!pip install -U google-cloud-aiplatform --user

In [4]:
PROJECT_ID = "esoteric-stream-399606"  # @param {type:"string"}
LOCATION = "us-central1" # @param {type:"string"}

In [7]:
from vertexai.preview.language_models import TextGenerationModel,\
                                            ChatModel,\
                                            InputOutputTextPair,\
                                            TextEmbeddingModel

from IPython.display import display, Markdown
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns

from google.colab import auth as google_auth
google_auth.authenticate_user()

import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION)

In [9]:
chat_model = ChatModel.from_pretrained("chat-bison@001")
print(type(chat_model))

chat = chat_model.start_chat()
print(type(chat))

print(chat.send_message("""
My name is Larry.
"""))

<class 'vertexai.preview.language_models._PreviewChatModel'>
<class 'vertexai.preview.language_models._PreviewChatSession'>
MultiCandidateTextGenerationResponse(text='Hi Larry, how can I help you today?', _prediction_response=Prediction(predictions=[{'safetyAttributes': [{'blocked': False, 'scores': [], 'categories': []}], 'candidates': [{'author': '1', 'content': 'Hi Larry, how can I help you today?'}], 'groundingMetadata': [None], 'citationMetadata': [{'citations': []}]}], deployed_model_id='', model_version_id='', model_resource_name='', explanations=None), is_blocked=False, safety_attributes={}, candidates=[Hi Larry, how can I help you today?])


In [19]:
print(chat.send_message("""
how to install 'google.cloud.sql.connector' module in colab?
""").text)

To install the 'google.cloud.sql.connector' module in colab, you can run the following command:

```
!pip install google-cloud-sql-connector
```

This will install the latest version of the module.


In [12]:
for m in chat.message_history :
  print(m.author, ':', m.content)
  print("")

user : 
My name is Larry.

bot : Hi Larry, how can I help you today?
user : 
What's my name again?

bot : Your name is Larry.
user : 
What's my name again?

bot : Your name is Larry.


In [13]:
chat = chat_model.start_chat(
    context="My name is Ned. You are my personal assistant. My favorite movies are Lord of the Rings and Hobbit.",
    examples=[
        InputOutputTextPair(
            input_text="Who do you work for?",
            output_text="I work for Ned.",
        ),
        InputOutputTextPair(
            input_text="What do I like?",
            output_text="Ned likes watching movies.",
        ),
    ],
    temperature=0.3,
    max_output_tokens=1024,
    top_p=0.8,
    top_k=1
)
print(chat.send_message("Are my favorite movies based on a book series?").text)

Yes, your favorite movies are based on a book series.


In [14]:
print(chat.send_message("When where these books published?").text)

The books were published in 1954 and 1955.


In [24]:
!pip install transformers
!pip install sqlalchemy
!pip install mysql-connector-python
import sys
!{sys.executable} -m pip install cloud-sql-python-connector["pg8000"] SQLAlchemy==2.0.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 8.1 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.22
    Uninstalling SQLAlchemy-2.0.22:
      Successfully uninstalled SQLAlchemy-2.0.22
  Attempting uninstall: pg8000
    Found existing installation: pg8000 1.30.3
    Uninstalling pg8000-1.30.3:
      Successfully uninstalled pg8000-1.30.3


In [26]:
import pg8000
from google.cloud.sql.connector import Connector, IPTypes
import sqlalchemy
from transformers import AutoModel, AutoTokenizer

def get_KoSimCSE():
    model = AutoModel.from_pretrained('BM-K/KoSimCSE-roberta-multitask')
    tokenizer = AutoTokenizer.from_pretrained('BM-K/KoSimCSE-roberta-multitask')

    print(type(model))
    print(type(tokenizer))

    return model, tokenizer

model, tokenizer = get_KoSimCSE()

<class 'transformers.models.roberta.modeling_roberta.RobertaModel'>
<class 'transformers.models.bert.tokenization_bert_fast.BertTokenizerFast'>


In [31]:
instance_connection_name = "esoteric-stream-399606:asia-northeast3:wjdfoek3"
db_user = "postgres"
db_pass = "pgvectorwjdfo"
db_name = "pgvector"

# initialize Cloud SQL Python Connector object
connector = Connector()

def getconn() -> pg8000.dbapi.Connection:
    conn: pg8000.dbapi.Connection = connector.connect(
        instance_connection_name,
        "pg8000",
        user=db_user,
        password=db_pass,
        db=db_name,
        ip_type=IPTypes.PUBLIC,
    )
    return conn

pool = sqlalchemy.create_engine(
    "postgresql+pg8000://",
    creator=getconn,
)


In [44]:
query = sqlalchemy.text(
"""
drop table empv3
"""
)

conn = pool.connect() # pool.connect()하고 한 transaction만 수행 가능함.
conn.begin()          # transaction 하나 수행하고 commit, close 하고 다시 connect 하기
try:
  conn.execute(query)
  print("empv3 table is dropped")
  conn.commit()
except : print("table \'empv3\' does not exists")
conn.close()

conn = pool.connect()
conn.begin()
query = sqlalchemy.text(
"""
create table empv3 (
ename varchar,
empno int primary key,
evec double precision[]
)
"""
)
conn.execute(query)
conn.commit()
conn.close()
print("empv3 table is created")

table 'empv3' does not exists
empv3 table is created


In [46]:
query = sqlalchemy.text(
"""
insert into empv3 values ('SMITH', 7369, '{1,2,3}');
insert into empv3 values ('JAMES', 7900, '{4,5,6}');
"""
)

conn = pool.connect()
conn.begin()
conn.execute(query)
conn.commit()
conn.close()
print("Insert Success")

Insert Success


In [47]:
query = sqlalchemy.text(
"""
select * from empv3;
"""
)

result = pool.connect().execute(query).fetchall()
print(result)

query = sqlalchemy.text(
"""
SELECT * FROM empv3 ORDER BY evec <-> '[3,4,5]' LIMIT 5;
"""
)

result = pool.connect().execute(query).fetchall()
print(result)


[('SMITH', 7369, [1.0, 2.0, 3.0]), ('JAMES', 7900, [4.0, 5.0, 6.0])]


DatabaseError: ignored

In [39]:
with pool.connect() as db_conn:
  db_conn.execute(
    sqlalchemy.text(
      "CREATE EXTENSION IF NOT EXISTS pg_trgm with schema public"
    )
  )
  db_conn.commit()

with pool.connect() as db_conn:

  db_conn.execute(sqlalchemy.text("DROP TABLE IF EXISTS empv4"))

  db_conn.execute(
    sqlalchemy.text(
      """
      create table empv4 (
        ename varchar,
        empno int primary key,
        org_text varchar(4096),
        evec vector(768)
      )
      """
    )
  )

  db_conn.commit()

DatabaseError: ignored

In [ ]:
sampel_text = [
    "안녕하세요",
    "반갑습니다",
    "안녕하세요 반갑습니다",
    "안녕하세요 반갑네요",
    "서울에 갑니다",
    "인천에 갑니다",
    "부산에 갑니다",
    "학교에 갑니다",
    "사무실에서 나옵니다",
    "사무실이 먼가요",
    "지하철을 탑니다",
    "버스를 탑니다",
]

In [ ]:
with pool.connect() as db_conn:

    empno = 5555
    for text in sampel_text:

        inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")

        embeddings, _ = model(**inputs, return_dict=False)
        embedding_arr = embeddings[0][0].detach().numpy()
        embedding_str = ",".join(str(x) for x in embedding_arr)
        embedding_str = "["+embedding_str+"]"

        insert_stmt = sqlalchemy.text(
            "INSERT INTO empv4 VALUES (:ename, :empno, :org_text, :evec)"
        )

        empno += 1
        ename = "emp"+str(empno)

        db_conn.execute(
            insert_stmt, parameters={"ename": ename, "empno": empno, "org_text": text, "evec": embedding_str}
        )

    db_conn.commit()

In [ ]:
with pool.connect() as db_conn:
    result = db_conn.execute(
        sqlalchemy.text(
            "SELECT * FROM empv4"
        ),
    ).fetchall()

for row in result:
    print(row[2])

In [ ]:
# query_text = "사무실 인사"
# query_text = "대구"
# query_text = "사무실에 갑니다"
# query_text = "사무실"
# query_text = "회사 위치"
query_text = "회사 인사"

inputs = tokenizer(query_text, padding=True, truncation=True, return_tensors="pt")

embeddings, _ = model(**inputs, return_dict=False)
embedding_arr = embeddings[0][0].detach().numpy()
embedding_str = ",".join(str(x) for x in embedding_arr)
embedding_str = "["+embedding_str+"]"

with pool.connect() as db_conn:
    result = db_conn.execute(
        sqlalchemy.text(
            "SELECT org_text FROM empv4 ORDER BY evec <-> :query_vec LIMIT 5"
        ),
        parameters={"query_vec": embedding_str},
    ).fetchall()

for row in result:
    print(row[0])